###Exercise 1

Imports + reproducibility

In [1]:
from pathlib import Path
import os
import hashlib

import numpy as np
import pandas as pd

import cv2
from sklearn.cluster import MiniBatchKMeans
import joblib

try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs):
        return x

RANDOM_SEED = 0
rng = np.random.default_rng(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


ModuleNotFoundError: No module named 'pandas'

Configuration (edit paths and parameters here)

In [ ]:
# ---- REQUIRED: set this to your local Caltech101 folder ----
# This should point to the directory that contains the category folders.
# Typical example:
# DATASET_ROOT = Path("/path/to/101_ObjectCategories")
DATASET_ROOT = Path("CHANGE_ME/101_ObjectCategories")

# Choose categories explicitly, or set to None to auto-pick the first N (excluding BACKGROUND_Google).
CATEGORIES = None
N_CATEGORIES_AUTO = 5

# Training/test split per category:
# - If you use a small subset, a common setting is 10 train + 10 test per category.
TRAIN_PER_CLASS = 10
TEST_PER_CLASS = 10

# Codebook size (k in k-means). Typical values: 200–500 for small setups.
K = 400

# Descriptor sampling for k-means training to keep runtime reasonable:
DESC_PER_IMAGE_FOR_KMEANS = 300         # take at most this many descriptors from each train image for k-means fitting
MAX_KMEANS_DESCRIPTORS = 120_000        # global cap for k-means fitting matrix

# Optional speed knob: cap descriptors used when building each image histogram (None = use all)
MAX_DESC_PER_IMAGE_FOR_HIST = None

# Normalize histograms (recommended)
L1_NORMALIZE_HIST = True

# Output artifacts
OUTPUT_DIR = Path("./exercise1_codebook_artifacts")
DESC_CACHE_DIR = OUTPUT_DIR / "desc_cache"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DESC_CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("DATASET_ROOT:", DATASET_ROOT.resolve())
print("OUTPUT_DIR:", OUTPUT_DIR.resolve())


Dataset utilities (scan categories, split train/test)

In [ ]:
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def list_categories(dataset_root: Path):
    if not dataset_root.exists():
        raise FileNotFoundError(
            f"DATASET_ROOT does not exist: {dataset_root}\n"
            "Set DATASET_ROOT to the Caltech101 category folder (e.g., .../101_ObjectCategories)."
        )
    cats = [p.name for p in dataset_root.iterdir() if p.is_dir()]
    # Common to exclude this category in Caltech101:
    cats = [c for c in cats if c.lower() != "background_google"]
    cats.sort()
    return cats

def list_images_for_category(cat_dir: Path):
    files = [p for p in cat_dir.iterdir() if p.is_file() and p.suffix.lower() in IMG_EXTS]
    files.sort()
    return files

def split_train_test(files, n_train, n_test, seed):
    files = list(files)
    local_rng = np.random.default_rng(seed)
    local_rng.shuffle(files)
    n_total = min(len(files), n_train + n_test)
    files = files[:n_total]
    train_files = files[:min(n_train, len(files))]
    test_files = files[min(n_train, len(files)):min(n_train + n_test, len(files))]
    # Ensure equal sizes if possible
    m = min(len(train_files), len(test_files))
    return train_files[:m], test_files[:m]

all_categories = list_categories(DATASET_ROOT)
if CATEGORIES is None:
    CATEGORIES = all_categories[:N_CATEGORIES_AUTO]
else:
    missing = [c for c in CATEGORIES if (DATASET_ROOT / c).is_dir() is False]
    if missing:
        raise ValueError(f"These categories were not found under DATASET_ROOT: {missing}")

print(f"Using {len(CATEGORIES)} categories:")
print(CATEGORIES)


Build a metadata table for the split (train/test)

In [ ]:
rows = []
for ci, cat in enumerate(CATEGORIES):
    cat_dir = DATASET_ROOT / cat
    files = list_images_for_category(cat_dir)
    train_files, test_files = split_train_test(
        files, n_train=TRAIN_PER_CLASS, n_test=TEST_PER_CLASS, seed=RANDOM_SEED + ci
    )

    for p in train_files:
        rows.append({"path": p, "category": cat, "split": "train"})
    for p in test_files:
        rows.append({"path": p, "category": cat, "split": "test"})

meta = pd.DataFrame(rows)
if meta.empty:
    raise RuntimeError("No images found. Check DATASET_ROOT and expected folder structure.")

print(meta.groupby(["category", "split"]).size().unstack(fill_value=0))
meta.head()


SIFT extractor + descriptor caching

In [ ]:
def create_sift():
    # OpenCV builds differ; try the standard entry point first.
    if hasattr(cv2, "SIFT_create"):
        return cv2.SIFT_create()
    # Older fallback (opencv-contrib)
    if hasattr(cv2, "xfeatures2d") and hasattr(cv2.xfeatures2d, "SIFT_create"):
        return cv2.xfeatures2d.SIFT_create()
    raise RuntimeError(
        "SIFT is not available in your OpenCV build.\n"
        "Install opencv-contrib-python (not just opencv-python), then restart the kernel."
    )

sift = create_sift()

def _cache_path_for_image(image_path: Path) -> Path:
    # Hash absolute path so the cache name is safe and unique.
    h = hashlib.md5(str(image_path.resolve()).encode("utf-8")).hexdigest()
    return DESC_CACHE_DIR / f"{h}.npz"

def read_gray(image_path: Path) -> np.ndarray:
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Failed to read image: {image_path}")
    return img

def get_sift_descriptors(image_path: Path, use_cache: bool = True) -> np.ndarray:
    cpath = _cache_path_for_image(image_path)
    if use_cache and cpath.exists():
        data = np.load(cpath)
        return data["desc"].astype(np.float32, copy=False)

    img = read_gray(image_path)
    kps, desc = sift.detectAndCompute(img, None)

    if desc is None:
        desc = np.empty((0, 128), dtype=np.float32)
    else:
        desc = desc.astype(np.float32, copy=False)

    if use_cache:
        np.savez_compressed(cpath, desc=desc)
    return desc


Collect descriptors from training images for k-means

In [ ]:
train_meta = meta[meta["split"] == "train"].reset_index(drop=True)
train_paths = train_meta["path"].tolist()

desc_blocks = []
total_added = 0

for p in tqdm(train_paths, desc="Extracting SIFT (train) for k-means"):
    desc = get_sift_descriptors(p, use_cache=True)
    if desc.shape[0] == 0:
        continue

    # Sample per-image for k-means fitting
    if desc.shape[0] > DESC_PER_IMAGE_FOR_KMEANS:
        idx = rng.choice(desc.shape[0], size=DESC_PER_IMAGE_FOR_KMEANS, replace=False)
        desc = desc[idx]

    desc_blocks.append(desc)
    total_added += desc.shape[0]

X = np.vstack(desc_blocks) if desc_blocks else np.empty((0, 128), dtype=np.float32)
print("Descriptors collected for k-means:", X.shape)

if X.shape[0] == 0:
    raise RuntimeError("No SIFT descriptors found in training images. Try different categories or check images.")

# Global cap
if X.shape[0] > MAX_KMEANS_DESCRIPTORS:
    idx = rng.choice(X.shape[0], size=MAX_KMEANS_DESCRIPTORS, replace=False)
    X = X[idx]
    print("After global cap:", X.shape)

X = X.astype(np.float32, copy=False)


Fit the visual-word codebook (k-means)

In [ ]:
# MiniBatchKMeans is typically much faster than full KMeans for large descriptor sets.
kmeans_kwargs = dict(
    n_clusters=K,
    random_state=RANDOM_SEED,
    batch_size=4096,
    max_iter=200,
    verbose=0
)

try:
    kmeans = MiniBatchKMeans(**kmeans_kwargs, n_init="auto")
except TypeError:
    # For older scikit-learn versions
    kmeans = MiniBatchKMeans(**kmeans_kwargs, n_init=3)

kmeans.fit(X)

print("Codebook learned.")
print("Cluster centers shape:", kmeans.cluster_centers_.shape)
print("Inertia:", float(kmeans.inertia_))


Build BoW histogram for each training image

In [ ]:
def bow_histogram(desc: np.ndarray, kmeans_model: MiniBatchKMeans, k: int, l1_normalize: bool = True) -> np.ndarray:
    hist = np.zeros((k,), dtype=np.float32)
    if desc is None or desc.shape[0] == 0:
        return hist

    words = kmeans_model.predict(desc)  # nearest centroid index for each descriptor
    hist = np.bincount(words, minlength=k).astype(np.float32)

    if l1_normalize:
        s = hist.sum()
        if s > 0:
            hist /= s
    return hist

bows = []
n_empty = 0

for p in tqdm(train_paths, desc="Building BoW histograms (train)"):
    desc = get_sift_descriptors(p, use_cache=True)

    if desc.shape[0] == 0:
        n_empty += 1
        bows.append(np.zeros((K,), dtype=np.float32))
        continue

    if MAX_DESC_PER_IMAGE_FOR_HIST is not None and desc.shape[0] > MAX_DESC_PER_IMAGE_FOR_HIST:
        idx = rng.choice(desc.shape[0], size=MAX_DESC_PER_IMAGE_FOR_HIST, replace=False)
        desc = desc[idx]

    bows.append(bow_histogram(desc, kmeans, K, l1_normalize=L1_NORMALIZE_HIST))

train_meta = train_meta.copy()
train_meta["bow"] = bows

print("Done.")
print("Training images:", len(train_meta))
print("Images with 0 descriptors:", n_empty)
train_meta.head()


Save outputs (codebook + training BoW)

In [ ]:
# Save k-means model (contains the codebook centers)
codebook_path = OUTPUT_DIR / f"codebook_k{K}_minibatchkmeans.joblib"
joblib.dump(kmeans, codebook_path)

# Save centers explicitly as numpy (optional but convenient)
centers_path = OUTPUT_DIR / f"codebook_centers_k{K}.npy"
np.save(centers_path, kmeans.cluster_centers_.astype(np.float32))

# Save BoW matrix + metadata for training images
B = np.vstack(train_meta["bow"].to_numpy()).astype(np.float32)
paths = train_meta["path"].astype(str).to_numpy()
labels = train_meta["category"].to_numpy()

bow_path = OUTPUT_DIR / f"train_bow_k{K}.npz"
np.savez_compressed(bow_path, B=B, paths=paths, labels=labels)

# Also save a simple CSV (without the raw vectors) for inspection
csv_path = OUTPUT_DIR / f"train_meta_k{K}.csv"
train_meta.drop(columns=["bow"]).to_csv(csv_path, index=False)

print("Saved:")
print(" -", codebook_path)
print(" -", centers_path)
print(" -", bow_path)
print(" -", csv_path)
print("BoW matrix shape:", B.shape)


Quick sanity checks (optional)

In [ ]:
# Check a few histograms sum to 1 (if L1 normalized)
sums = B.sum(axis=1)
print("Histogram sum stats:")
print(" min:", float(sums.min()), " max:", float(sums.max()), " mean:", float(sums.mean()))

# Count per-class training images
print("\nTrain images per class:")
print(pd.Series(labels).value_counts())


###Exercise 2

Prerequisite check + (optional) load codebook if needed

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import joblib

# Expect these from Exercise 1:
# - meta : DataFrame with columns ["path", "category", "split"]
# - K : codebook size (int)
# - OUTPUT_DIR : output folder Path
# - get_sift_descriptors : function (path -> (N,128) float32)
# - kmeans : fitted (MiniBatch)KMeans with .predict()

missing = []
for name in ["meta", "K", "OUTPUT_DIR"]:
    if name not in globals():
        missing.append(name)
if missing:
    raise RuntimeError(
        "Missing required variables from Exercise 1: "
        + ", ".join(missing)
        + "\nRun the Exercise 1 cells first (dataset split + codebook learning)."
    )

# Load codebook model from disk if not present in memory
if "kmeans" not in globals():
    codebook_path = Path(OUTPUT_DIR) / f"codebook_k{K}_minibatchkmeans.joblib"
    if not codebook_path.exists():
        raise FileNotFoundError(
            f"Could not find saved codebook at: {codebook_path}\n"
            "Either re-run Exercise 1 (k-means fitting) or update OUTPUT_DIR/K."
        )
    kmeans = joblib.load(codebook_path)

# Ensure descriptor extraction exists; if not, stop with a clear message
if "get_sift_descriptors" not in globals():
    raise RuntimeError(
        "get_sift_descriptors(...) is not defined.\n"
        "Run the Exercise 1 cell that defines SIFT extraction + caching."
    )

print("OK: meta/K/OUTPUT_DIR present, and codebook (kmeans) is available.")
print("Indexing images:", len(meta), "| K =", K)


BoW construction (counts + L1-normalized)

In [ ]:
def bow_counts_and_l1(desc: np.ndarray, kmeans_model, k: int):
    """
    Returns:
      hist_counts: (k,) float32
      hist_l1:     (k,) float32 (L1-normalized, or all zeros if no descriptors)
    """
    hist_counts = np.zeros((k,), dtype=np.float32)

    if desc is None or desc.shape[0] == 0:
        return hist_counts, hist_counts.copy()

    word_ids = kmeans_model.predict(desc)  # nearest centroid index per descriptor
    hist_counts = np.bincount(word_ids, minlength=k).astype(np.float32)

    hist_l1 = hist_counts.copy()
    s = float(hist_l1.sum())
    if s > 0:
        hist_l1 /= s
    return hist_counts, hist_l1


Compute BoW for every image (train + test) and build the index table

In [ ]:
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

index_df = meta.copy().reset_index(drop=True)
index_df["filename"] = index_df["path"].apply(lambda p: Path(p).name)

bows_counts = []
bows_l1 = []
n_empty = 0

for p in tqdm(index_df["path"].tolist(), desc="Indexing images (SIFT -> BoW)"):
    desc = get_sift_descriptors(Path(p), use_cache=True)

    if desc.shape[0] == 0:
        n_empty += 1

    hc, hl1 = bow_counts_and_l1(desc, kmeans, K)
    bows_counts.append(hc)
    bows_l1.append(hl1)

index_df["bow_counts"] = bows_counts
index_df["bow_l1"] = bows_l1

print("Done indexing.")
print("Images:", len(index_df))
print("Images with 0 descriptors:", n_empty)

index_df.head()


Save the index (metadata table + feature matrices) for repeated reuse

In [ ]:
OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Stack to matrices for fast retrieval later
B_counts = np.vstack(index_df["bow_counts"].to_numpy()).astype(np.float32)
B_l1 = np.vstack(index_df["bow_l1"].to_numpy()).astype(np.float32)

paths = index_df["path"].astype(str).to_numpy()
filenames = index_df["filename"].astype(str).to_numpy()
labels = index_df["category"].astype(str).to_numpy()
splits = index_df["split"].astype(str).to_numpy()

# Save metadata (human-readable)
index_meta_path = OUTPUT_DIR / f"index_meta_k{K}.csv"
index_df.drop(columns=["bow_counts", "bow_l1"]).to_csv(index_meta_path, index=False)

# Save features (compact + fast to load)
index_features_path = OUTPUT_DIR / f"index_features_k{K}.npz"
np.savez_compressed(
    index_features_path,
    B_counts=B_counts,
    B_l1=B_l1,
    paths=paths,
    filenames=filenames,
    labels=labels,
    splits=splits,
    K=np.array([K], dtype=np.int32),
)

# Optional: save a single pickle/joblib if you prefer one file (not required)
index_joblib_path = OUTPUT_DIR / f"index_table_k{K}.joblib"
joblib.dump(index_df, index_joblib_path)

print("Saved indexing artifacts:")
print(" -", index_meta_path)
print(" -", index_features_path)
print(" -", index_joblib_path)
print("B_counts shape:", B_counts.shape, "| B_l1 shape:", B_l1.shape)


Convenience loader (use in Exercise 3)

In [ ]:
def load_index_table(output_dir: Path, k: int):
    output_dir = Path(output_dir)
    meta_path = output_dir / f"index_meta_k{k}.csv"
    feat_path = output_dir / f"index_features_k{k}.npz"

    if not meta_path.exists() or not feat_path.exists():
        raise FileNotFoundError(
            "Missing index files.\n"
            f"Expected:\n  {meta_path}\n  {feat_path}"
        )

    meta_df = pd.read_csv(meta_path)
    z = np.load(feat_path, allow_pickle=False)
    B_counts = z["B_counts"].astype(np.float32, copy=False)
    B_l1 = z["B_l1"].astype(np.float32, copy=False)

    return meta_df, B_counts, B_l1

# Example usage:
meta_df_loaded, B_counts_loaded, B_l1_loaded = load_index_table(OUTPUT_DIR, K)
print(meta_df_loaded.head())
print("Loaded shapes:", B_counts_loaded.shape, B_l1_loaded.shape)


###Exercise 3

Cell 1 — Load the saved index (or reuse index_df if already in memory)

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

# Requires from previous exercises:
# - OUTPUT_DIR (Path or str)
# - K (int)

if "OUTPUT_DIR" not in globals() or "K" not in globals():
    raise RuntimeError("Missing OUTPUT_DIR or K. Run Exercise 1/2 cells first.")

OUTPUT_DIR = Path(OUTPUT_DIR)

def load_index_features(output_dir: Path, k: int):
    feat_path = output_dir / f"index_features_k{k}.npz"
    meta_path = output_dir / f"index_meta_k{k}.csv"

    if not feat_path.exists():
        raise FileNotFoundError(f"Missing: {feat_path}. Run Exercise 2 saving cell first.")
    if not meta_path.exists():
        raise FileNotFoundError(f"Missing: {meta_path}. Run Exercise 2 saving cell first.")

    z = np.load(feat_path, allow_pickle=False)
    B_counts = z["B_counts"].astype(np.float32, copy=False)
    B_l1 = z["B_l1"].astype(np.float32, copy=False)
    paths = z["paths"].astype(str)
    labels = z["labels"].astype(str)
    splits = z["splits"].astype(str)
    meta_df = pd.read_csv(meta_path)

    return meta_df, B_counts, B_l1, paths, labels, splits

# Prefer in-memory index_df if present and well-formed, otherwise load from disk.
use_in_memory = ("index_df" in globals() and isinstance(index_df, pd.DataFrame)
                 and "bow_counts" in index_df.columns and "bow_l1" in index_df.columns)

if use_in_memory:
    paths_all = index_df["path"].astype(str).to_numpy()
    labels_all = index_df["category"].astype(str).to_numpy()
    splits_all = index_df["split"].astype(str).to_numpy()
    B_counts_all = np.vstack(index_df["bow_counts"].to_numpy()).astype(np.float32)
    B_l1_all = np.vstack(index_df["bow_l1"].to_numpy()).astype(np.float32)
    meta_df = index_df.drop(columns=["bow_counts", "bow_l1"]).copy()
else:
    meta_df, B_counts_all, B_l1_all, paths_all, labels_all, splits_all = load_index_features(OUTPUT_DIR, K)

train_mask = (splits_all == "train")
test_mask  = (splits_all == "test")

print("All images:", len(paths_all))
print("Train:", int(train_mask.sum()), "| Test:", int(test_mask.sum()), "| K:", K)


Prepare representations used by the similarity measures

In [ ]:
# Training database (used in both experiments)
D_train_counts = B_counts_all[train_mask]
D_train_l1     = B_l1_all[train_mask]
D_train_paths  = paths_all[train_mask]
D_train_labels = labels_all[train_mask]

# Queries for the two experiments
Q_train_counts = D_train_counts
Q_train_l1     = D_train_l1
Q_train_paths  = D_train_paths
Q_train_labels = D_train_labels

Q_test_counts  = B_counts_all[test_mask]
Q_test_l1      = B_l1_all[test_mask]
Q_test_paths   = paths_all[test_mask]
Q_test_labels  = labels_all[test_mask]

# 1) Common-words presence vectors (boolean)
D_train_bin = (D_train_counts > 0)
Q_train_bin = (Q_train_counts > 0)
Q_test_bin  = (Q_test_counts  > 0)

# 2) Bhattacharyya: sqrt of L1 histograms (still valid as long as entries are nonnegative)
D_train_sqrt = np.sqrt(np.clip(D_train_l1, 0.0, None))
Q_train_sqrt = np.sqrt(np.clip(Q_train_l1, 0.0, None))
Q_test_sqrt  = np.sqrt(np.clip(Q_test_l1,  0.0, None))

# 3) TF-IDF (IDF computed on TRAINING set only)
df = D_train_bin.sum(axis=0).astype(np.float32)     # document frequency per word
N  = float(D_train_bin.shape[0])
idf = (np.log((N + 1.0) / (df + 1.0)) + 1.0).astype(np.float32)  # (K,)

def tfidf_l2_normalize(B_l1: np.ndarray, idf: np.ndarray) -> np.ndarray:
    V = (B_l1 * idf[None, :]).astype(np.float32, copy=False)
    norms = np.linalg.norm(V, axis=1, keepdims=True)
    mask = norms[:, 0] > 0
    Vn = np.zeros_like(V, dtype=np.float32)
    Vn[mask] = V[mask] / norms[mask]
    return Vn

D_train_tfidf = tfidf_l2_normalize(D_train_l1, idf)
Q_train_tfidf = D_train_tfidf
Q_test_tfidf  = tfidf_l2_normalize(Q_test_l1, idf)

print("Prepared: common-words, Bhattacharyya, TF-IDF.")


Similarity score functions

In [ ]:
def scores_common_words(Q_bin: np.ndarray, D_bin: np.ndarray) -> np.ndarray:
    # Similarity = count of indices where both have presence True
    # bool @ bool -> int; cast to float for uniform downstream handling
    return (Q_bin @ D_bin.T).astype(np.float32)

def scores_cosine(Q_l2: np.ndarray, D_l2: np.ndarray) -> np.ndarray:
    # Assumes rows are L2-normalized; similarity is dot product
    return (Q_l2 @ D_l2.T).astype(np.float32)

def scores_bhattacharyya(Q_sqrt: np.ndarray, D_sqrt: np.ndarray) -> np.ndarray:
    # Bhattacharyya coefficient for L1-normalized histograms:
    # BC(p,q) = sum_j sqrt(p_j q_j) = <sqrt(p), sqrt(q)>
    return (Q_sqrt @ D_sqrt.T).astype(np.float32)

def prepare_kl_db(D_l1: np.ndarray, eps: float = 1e-10):
    D = (D_l1.astype(np.float32, copy=False) + np.float32(eps))
    D /= D.sum(axis=1, keepdims=True)
    logD = np.log(D)
    row_D_logD = np.sum(D * logD, axis=1)  # shape (n_db,)
    return D, logD, row_D_logD

def scores_sym_kl(Q_l1: np.ndarray, D_prepared, eps: float = 1e-10, batch_size: int = 128) -> np.ndarray:
    """
    Returns similarity = - symmetric KL distance.
    """
    D, logD, row_D_logD = D_prepared
    n_q = Q_l1.shape[0]
    n_db = D.shape[0]
    out = np.empty((n_q, n_db), dtype=np.float32)

    for start in range(0, n_q, batch_size):
        end = min(start + batch_size, n_q)
        Q = (Q_l1[start:end].astype(np.float32, copy=False) + np.float32(eps))
        Q /= Q.sum(axis=1, keepdims=True)
        logQ = np.log(Q)

        # KL(Q || D): const(Q) - logD @ Q^T
        const = np.sum(Q * logQ, axis=1)  # (b,)
        term_logD = (logD @ Q.T).T        # (b, n_db)
        kl_qd = const[:, None] - term_logD

        # KL(D || Q): row(D log D) - D @ logQ^T
        term_DlogQ = (D @ logQ.T).T       # (b, n_db)
        kl_dq = row_D_logD[None, :] - term_DlogQ

        sym = 0.5 * (kl_qd + kl_dq)       # (b, n_db)
        out[start:end] = (-sym).astype(np.float32)  # similarity = -distance

    return out


Ranking + metrics (MRR and Top-3)

In [ ]:
def evaluate_retrieval(scores: np.ndarray,
                       query_labels: np.ndarray,
                       db_labels: np.ndarray,
                       topk: int = 3,
                       exclude_self_diag: bool = False):
    """
    scores: (n_q, n_db), higher is better
    """
    S = scores.copy()
    if exclude_self_diag:
        if S.shape[0] != S.shape[1]:
            raise ValueError("exclude_self_diag=True requires a square score matrix.")
        np.fill_diagonal(S, -np.inf)

    n_q = S.shape[0]
    rr = np.zeros((n_q,), dtype=np.float32)
    topk_hit = np.zeros((n_q,), dtype=np.float32)
    first_ranks = np.full((n_q,), np.inf, dtype=np.float32)

    for i in range(n_q):
        order = np.argsort(-S[i])  # descending similarity
        lbls = db_labels[order]

        hits = np.where(lbls == query_labels[i])[0]
        if hits.size > 0:
            rank = float(hits[0] + 1)  # 1-indexed
            first_ranks[i] = rank
            rr[i] = 1.0 / rank

        topk_hit[i] = 1.0 if np.any(lbls[:topk] == query_labels[i]) else 0.0

    mrr = float(rr.mean()) if n_q > 0 else 0.0
    topk_pct = float(100.0 * topk_hit.mean()) if n_q > 0 else 0.0
    return {
        "MRR": mrr,
        "Top3_%": topk_pct,
        "ranks": first_ranks,
    }


Run both required experiments for multiple similarity measures

In [ ]:
# Precompute KL DB (training set only)
D_train_kl_prepared = prepare_kl_db(D_train_l1, eps=1e-10)

results = []

# ---- 1) Common words ----
S_train = scores_common_words(Q_train_bin, D_train_bin)
m_train = evaluate_retrieval(S_train, Q_train_labels, D_train_labels, topk=3, exclude_self_diag=True)

S_test = scores_common_words(Q_test_bin, D_train_bin)
m_test = evaluate_retrieval(S_test, Q_test_labels, D_train_labels, topk=3, exclude_self_diag=False)

results.append({
    "measure": "common_words",
    "train_MRR": m_train["MRR"], "train_Top3_%": m_train["Top3_%"],
    "test_MRR":  m_test["MRR"],  "test_Top3_%":  m_test["Top3_%"],
})

# ---- 2) TF-IDF cosine ----
S_train = scores_cosine(Q_train_tfidf, D_train_tfidf)
m_train = evaluate_retrieval(S_train, Q_train_labels, D_train_labels, topk=3, exclude_self_diag=True)

S_test = scores_cosine(Q_test_tfidf, D_train_tfidf)
m_test = evaluate_retrieval(S_test, Q_test_labels, D_train_labels, topk=3, exclude_self_diag=False)

results.append({
    "measure": "tfidf_cosine",
    "train_MRR": m_train["MRR"], "train_Top3_%": m_train["Top3_%"],
    "test_MRR":  m_test["MRR"],  "test_Top3_%":  m_test["Top3_%"],
})

# ---- 3) Bhattacharyya coefficient ----
S_train = scores_bhattacharyya(Q_train_sqrt, D_train_sqrt)
m_train = evaluate_retrieval(S_train, Q_train_labels, D_train_labels, topk=3, exclude_self_diag=True)

S_test = scores_bhattacharyya(Q_test_sqrt, D_train_sqrt)
m_test = evaluate_retrieval(S_test, Q_test_labels, D_train_labels, topk=3, exclude_self_diag=False)

results.append({
    "measure": "bhattacharyya",
    "train_MRR": m_train["MRR"], "train_Top3_%": m_train["Top3_%"],
    "test_MRR":  m_test["MRR"],  "test_Top3_%":  m_test["Top3_%"],
})

# ---- 4) Symmetric KL (similarity = -distance) ----
S_train = scores_sym_kl(Q_train_l1, D_train_kl_prepared, eps=1e-10, batch_size=128)
m_train = evaluate_retrieval(S_train, Q_train_labels, D_train_labels, topk=3, exclude_self_diag=True)

S_test = scores_sym_kl(Q_test_l1, D_train_kl_prepared, eps=1e-10, batch_size=128)
m_test = evaluate_retrieval(S_test, Q_test_labels, D_train_labels, topk=3, exclude_self_diag=False)

results.append({
    "measure": "sym_kl",
    "train_MRR": m_train["MRR"], "train_Top3_%": m_train["Top3_%"],
    "test_MRR":  m_test["MRR"],  "test_Top3_%":  m_test["Top3_%"],
})

results_df = pd.DataFrame(results).sort_values(by="test_MRR", ascending=False).reset_index(drop=True)
results_df


Save the results table (for your report)

In [ ]:
out_csv = OUTPUT_DIR / f"exercise3_results_k{K}.csv"
results_df.to_csv(out_csv, index=False)
print("Saved:", out_csv)


Visualize retrieval examples (optional, useful for the report)

In [ ]:
import matplotlib.pyplot as plt
import cv2

def _read_rgb(path: str):
    im = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if im is None:
        raise FileNotFoundError(f"Could not read: {path}")
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im

def show_retrieval(query_path: str, query_label: str,
                   db_paths: np.ndarray, db_labels: np.ndarray,
                   scores_row: np.ndarray, topk: int = 5,
                   title: str = ""):
    order = np.argsort(-scores_row)[:topk]
    fig = plt.figure(figsize=(3 * (topk + 1), 3))

    # Query
    ax = fig.add_subplot(1, topk + 1, 1)
    ax.imshow(_read_rgb(query_path))
    ax.set_title(f"Query\n{query_label}")
    ax.axis("off")

    # Top-k results
    for j, idx in enumerate(order, start=2):
        ax = fig.add_subplot(1, topk + 1, j)
        ax.imshow(_read_rgb(db_paths[idx]))
        ax.set_title(f"Rank {j-1}\n{db_labels[idx]}")
        ax.axis("off")

    if title:
        fig.suptitle(title)
    plt.show()

def get_score_row(metric: str, q_idx: int, split: str = "test"):
    if split == "test":
        q_path, q_lbl = Q_test_paths[q_idx], Q_test_labels[q_idx]
        q_bin, q_tfidf, q_sqrt, q_l1 = Q_test_bin[q_idx:q_idx+1], Q_test_tfidf[q_idx:q_idx+1], Q_test_sqrt[q_idx:q_idx+1], Q_test_l1[q_idx:q_idx+1]
    elif split == "train":
        q_path, q_lbl = Q_train_paths[q_idx], Q_train_labels[q_idx]
        q_bin, q_tfidf, q_sqrt, q_l1 = Q_train_bin[q_idx:q_idx+1], Q_train_tfidf[q_idx:q_idx+1], Q_train_sqrt[q_idx:q_idx+1], Q_train_l1[q_idx:q_idx+1]
    else:
        raise ValueError("split must be 'train' or 'test'")

    if metric == "common_words":
        s = scores_common_words(q_bin, D_train_bin)[0]
    elif metric == "tfidf_cosine":
        s = scores_cosine(q_tfidf, D_train_tfidf)[0]
    elif metric == "bhattacharyya":
        s = scores_bhattacharyya(q_sqrt, D_train_sqrt)[0]
    elif metric == "sym_kl":
        s = scores_sym_kl(q_l1, D_train_kl_prepared, eps=1e-10, batch_size=1)[0]
    else:
        raise ValueError(f"Unknown metric: {metric}")

    return q_path, q_lbl, s

# Example: show retrieval for a random test query using the best metric from results_df
if len(Q_test_paths) > 0:
    metric_best = results_df.loc[0, "measure"]
    q_idx = 0  # change to any index in [0, len(Q_test_paths)-1]
    q_path, q_lbl, srow = get_score_row(metric_best, q_idx, split="test")
    show_retrieval(q_path, q_lbl, D_train_paths, D_train_labels, srow, topk=5,
                   title=f"Test query retrieval using {metric_best}")


Simple plot for comparing measures (optional)

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(len(results_df))
fig = plt.figure(figsize=(10, 4))
plt.bar(x - 0.2, results_df["train_MRR"], width=0.4, label="Train retrieval MRR")
plt.bar(x + 0.2, results_df["test_MRR"],  width=0.4, label="Test classification MRR")
plt.xticks(x, results_df["measure"], rotation=20)
plt.ylabel("MRR")
plt.legend()
plt.tight_layout()
plt.show()
